# __WATER MLIP MD simulations__
Objective: generate enough sampling to determine if the experimental solvent-dependence of COSAN is observed using three different uMLIP models: UMA-small, MACE-MP-0b2 and ORB-v3. 

Secondary objective: See if in sampling the lack of rotational invariance of ORB models is diffuminated. Inspired by the ETH Zürich team regarding rotational invariance and direct force prediction. 

### __Model definitions__

In [1]:
# MACE
from mace.calculators import mace_mp

macemp = mace_mp(
    model="medium", 
    dispersion=False, 
    default_dtype="float32", 
    device='cpu',
    )

CALC_NAME = 'MACE-MP'

/Users/sergiortizropero/miniconda3/envs/ASE_all/lib/python3.10/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indice

cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.
Using Materials Project MACE for MACECalculator with /Users/sergiortizropero/.cache/mace/20231203mace128L1_epoch199model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.


/Users/sergiortizropero/miniconda3/envs/ASE_all/lib/python3.10/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_p

In [1]:
# ORB
from orb_models.forcefield.pretrained import orb_v3_conservative_inf_omat, orb_v2
from orb_models.forcefield.calculator import ORBCalculator

orbff_v2 = orb_v2(device='cpu')
orb_calc_v2 = ORBCalculator(orbff_v2, device='cpu')

#orbff_v3 = orb_v3_conservative_inf_omat(device='cpu')
#orbff_v3_calc = ORBCalculator(orbff_v3, device='cpu')

CALC_NAME = 'ORB'

/Users/sergiortizropero/miniconda3/envs/ASE_orbital/lib/python3.10/site-packages/orb_models/utils.py:30: UserWarning: Setting global torch default dtype to torch.float32.
  warnings.warn(f"Setting global torch default dtype to {torch_dtype}.")


In [ ]:
# UMA
from fairchem.core import FAIRChemCalculator
from fairchem.core.units.mlip_unit import load_predict_unit

# define the UMA ASE calculator
uma_predictor = load_predict_unit(
    path='/Users/sergiortizropero/TFG_phys/NNPs_TFG/models/ase_uma/uma-s-1p1.pt', 
    device='cpu',                   
    inference_settings='default',   
)
uma_calc = FAIRChemCalculator(
    uma_predictor,
    task_name='omol',               
)

CALC_NAME = 'UMA'

/Users/sergiortizropero/miniconda3/envs/ASE_UMA_test/lib/python3.12/site-packages/torchtnt/utils/version.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
W0718 11:51:27.198000 84253 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


### __Imports & Functions__

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
rc('lines', lw=1, color='b')
rc('legend', loc='best')
plt.rcParams["legend.fancybox"] = False
plt.rcParams["legend.edgecolor"] = 'black'
plt.rcParams['legend.borderpad'] = 0.25
plt.rcParams['legend.fontsize'] = 11
plt.rcParams.update({'pgf.preamble': r'\usepackage{amsmath}'})

from ase.visualize import view
from ase.build import molecule
from ase.io import write, read
from ase.io import Trajectory, trajectory
from ase import build
from ase.data.pubchem import pubchem_atoms_search

import ase.units as units
from ase import Atoms
from ase.calculators.tip3p import TIP3P, angleHOH, rOH
from ase.constraints import FixBondLengths
from ase.io.trajectory import Trajectory
from ase.io.trajectory import TrajectoryReader
from ase.md import Langevin, VelocityVerlet
from ase.optimize import BFGS
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.calculators.tip3p import angleHOH, rOH
import pickle
import ase.io
from tqdm import tqdm

try:
    import cmocean
    cmap = cmocean.cm.balance   
except:
    cmap = 'seismic'

In [3]:
def prepare_water(save_path, sim_name, T_0, pbc=False, box_size=None):

    x = angleHOH * np.pi / 180 / 2
    pos = [
        [0, 0, 0],
        [0, rOH * np.cos(x), rOH * np.sin(x)],
        [0, rOH * np.cos(x), -rOH * np.sin(x)],
    ]
    atoms = Atoms('OH2', positions=pos)
    

    # add calculator
    if CALC_NAME == 'MACE-MP':
        atoms.calc = macemp
    elif CALC_NAME == 'ORB':
        #atoms.calc = orbff_v3_calc
        atoms.calc = orb_calc_v2
    elif CALC_NAME == 'UMA':
        atoms.calc = uma_calc
    else:
        raise ValueError('name of the calculation not identified!')

    # system preprocessing
    if box_size is not None:
        atoms.set_cell(box_size)
    else:
        atoms.set_cell((20, 20, 20))
    atoms.center()

    if pbc:
        atoms.set_pbc(True)

    # path setup
    sim_path = os.path.join(save_path, sim_name)

    if not os.path.exists(sim_path):
        os.makedirs(sim_path)

    # initialize velocities
    MaxwellBoltzmannDistribution(atoms, temperature_K=T_0)

    return atoms


def load_water_box(save_path, sim_name, T_0, pbc=False, box_size=None):

    
    water_box_pdb = '/Users/sergiortizropero/TFG_phys/COSAN_MLIP/simulations/water_mds/water_box.pdb'
    atoms = read(water_box_pdb)
    atoms.info = {
        'spin': 1,
        'charge': 0,
        'k': 0,
    }

    # add calculator
    if CALC_NAME == 'MACE-MP':
        atoms.calc = macemp
    elif CALC_NAME == 'ORB':
        #atoms.calc = orbff_v3_calc
        atoms.calc = orb_calc_v2
    elif CALC_NAME == 'UMA':
        atoms.calc = uma_calc
    else:
        raise ValueError('name of the calculation not identified!')

    # system preprocessing
    if box_size is not None:
        atoms.set_cell(box_size)
    else:
        atoms.set_cell((20, 20, 20))
    atoms.center()

    if pbc:
        atoms.set_pbc(True)

    # path setup
    sim_path = os.path.join(save_path, sim_name)

    if not os.path.exists(sim_path):
        os.makedirs(sim_path)

    # initialize velocities
    MaxwellBoltzmannDistribution(atoms, temperature_K=T_0)

    return atoms




def run_NVT(atoms, length, dt, T, sim_path, sim_name, k, save_int=100, overwrite=False):

    # define paths
    log_path = os.path.join(sim_path, sim_name + f'_{k}.log')
    traj_path = os.path.join(sim_path, sim_name + f'_{k}.traj')

    # NVT run
    md = Langevin(
        atoms,
        dt*units.fs,
        temperature_K=T,
        friction=0.01,
        logfile=log_path,
        loginterval=save_int,
    )

    # trajectory
    if os.path.exists(traj_path) and (not overwrite):
        raise ValueError('Trajectory file already exists!')
    traj = Trajectory(traj_path, 'w', atoms)

    # save every save_int steps
    md.attach(traj.write, interval=save_int)
    md.run(length)
    print(f'md {k} has finished')

    return atoms


def restart_md(pkl_path, verbose=True):
    
    with open(pkl_path, 'rb') as f:
        cosan_restart = pickle.load(f)

    if verbose:
        print('restarting with Atoms.object with properties:')
        print(cosan_restart.info['charge'])
        print(cosan_restart.info['spin'])
        print(cosan_restart.info['k'])
        print(cosan_restart.info['positions'])
        print(cosan_restart.info['velocities'])

    return cosan_restart



def relax_structure(atoms, sim_path, convergence=2):
    dyn = BFGS(atoms, trajectory=os.path.join(sim_path, 'relaxation.traj'))
    dyn.run(fmax=convergence)

# __WATER MD__

In [4]:
K = 50              # number of fragments
length = 100000     # in steps
dt = 1              # in fs
T_init = 300.       # in K
T_target = 300.     # in K
save_int = 100

ps_length = float(K * length * dt / 1000)
sim_name = f'single_{CALC_NAME}_{ps_length:.1f}'
save_path = '/Users/sergiortizropero/TFG_phys/COSAN_MLIP/simulations/water_mds'
sim_path = os.path.join(save_path, sim_name)

restart = True
restart_file = '/Users/sergiortizropero/TFG_phys/COSAN_MLIP/simulations/water_mds/single_ORB_1000.0/single_ORB_1000.0_10_water.pkl'

print(f'restarting: {restart}')
print(sim_name)
print('Performing WATER {:.1f} ps NVT MD @ {} K'.format(ps_length, T_target))

restarting: True
single_ORB_5000.0
Performing WATER 5000.0 ps NVT MD @ 300.0 K


In [6]:
if not restart:
    # build the waters
    water = prepare_water( 
        save_path, 
        sim_name, 
        T_init,
    )
    water.info = {
        'spin': 1,
        'charge': 0,
        'k': 0,
    }
    relax_structure(water, sim_path, convergence=0.01)

else:
    # retreive latest pickle.
    water = restart_md(restart_file, verbose=True)    


converged = False
while not converged: 

    # start with k = 0
    # add 1
    water.info['k'] += 1
    k = water.info['k']

    # run a fragment
    # simulate k = 1
    run_NVT(
        water, 
        length, 
        dt, 
        T=T_target, 
        sim_path=sim_path, 
        sim_name=sim_name, 
        k=k, 
        save_int=save_int, 
        overwrite=restart,
    )

    # save to a file
    water.info['positions'] = water.get_positions()
    water.info['velocities'] = water.get_velocities()
    print(water.info['positions'])
    print(water.info['velocities'])
    print(water.info['k'])

    with open(os.path.join(sim_path, sim_name+f'_{k}_water.pkl'), 'wb') as f:
        pickle.dump(water, f)

    # check convergence
    if water.info['k'] >= K:
        converged = True

restarting with Atoms.object with properties:
0
1
10
[[10.12670523  0.5095077  14.47819264]
 [10.0591151   0.5563423  13.48180558]
 [11.12022103  0.55724711 14.56352142]]
[[-0.00416968 -0.02429835 -0.02626098]
 [-0.23484894  0.13895955  0.36357402]
 [ 0.3010299   0.24670466  0.05324104]]
md 11 has finished
[[10.1715164   0.45637737 14.37903832]
 [ 9.64721515  1.24086244 14.32305236]
 [10.58731963  0.57724932 15.27183363]]
[[ 0.03158546  0.01192435 -0.00659585]
 [-0.35486472 -0.12940763  0.23656062]
 [-0.14646018 -0.05985577 -0.13187128]]
11
md 12 has finished
[[10.18495499  0.49314434 14.4306858 ]
 [10.24278427  0.30366087 15.37142937]
 [10.56653281  1.39005552 14.38736669]]
[[-1.19493746e-04 -3.18045808e-02  2.90815580e-02]
 [-1.26795108e-01  2.59863432e-01 -3.73671643e-01]
 [ 1.28691410e-01  2.44939461e-01 -8.79115473e-02]]
12
md 13 has finished
[[10.20079843  0.55701125 14.5438676 ]
 [10.38315992 -0.31424473 14.91039212]
 [ 9.8699143   0.23976576 13.74350603]]
[[-0.03593886  0.02400

KeyboardInterrupt: 

# __WATER BOX MD__

In [39]:
K = 1              # number of fragments
length = 1000       # in steps
dt = 1              # in fs
T_init = 300.       # in K
T_target = 300.     # in K
save_int = 100


ps_length = float(K * length * dt / 1000)
sim_name = f'solvated_{CALC_NAME}_{ps_length:.1f}'
#sim_name = 'test'
save_path = '/Users/sergiortizropero/TFG_phys/NNPs_TFG/COSAN/simulations/water_mds'
sim_path = os.path.join(save_path, sim_name)

# build the waters
waters = load_water_box( 
    save_path, 
    sim_name, 
    T_init,
    #box_size=(18.774, 18.774, 18.774),
    box_size=(20, 20, 20),
    pbc=True,
)
waters.info = {
    'spin': 1,
    'charge': 0,
    'k': 0,
}

restart = False
restart_file = ''

print(f'restarting: {restart}')
print(sim_name)
print('Performing WATER {:.1f} ps NVT MD @ {} K'.format(ps_length, T_target))

restarting: False
solvated_ORB_1.0
Performing WATER 1.0 ps NVT MD @ 300.0 K


In [40]:
relax_structure(waters, sim_path)
with open(os.path.join(sim_path, 'optimized.pkl'), 'wb') as f:
        pickle.dump(waters, f)

if restart:
    # retreive latest pickle.
    waters = restart_md(restart_file, verbose=True)

converged = False
while not converged: 

    # start with k = 0
    # add 1
    waters.info['k'] += 1
    k = waters.info['k']

    # run a fragment
    # simulate k = 1
    run_NVT(
        waters, 
        length, 
        dt, 
        T=T_target, 
        sim_path=sim_path, 
        sim_name=sim_name, 
        k=k, 
        save_int=save_int, 
        overwrite=restart,
    )

    # save to a file
    waters.info['positions'] = waters.get_positions()
    waters.info['velocities'] = waters.get_velocities()
    print(waters.info['positions'])
    print(waters.info['velocities'])
    print(waters.info['k'])

    with open(os.path.join(sim_path, sim_name+f'_{k}_water.pkl'), 'wb') as f:
        pickle.dump(waters, f)

    # check convergence
    if waters.info['k'] >= K:
        converged = True

      Step     Time          Energy          fmax
BFGS:    0 18:36:40    -2445.266113      192.170619
BFGS:    1 18:36:42    -2769.355469       68.290928
BFGS:    2 18:36:44    -3073.040527       43.921440
BFGS:    3 18:36:46    -3309.744873       57.859860
BFGS:    4 18:36:49    -3415.162109       22.183399
BFGS:    5 18:36:51    -3509.240723       10.359201
BFGS:    6 18:36:53    -3573.539551       12.053555
BFGS:    7 18:36:55    -3610.803711        6.180288
BFGS:    8 18:36:57    -3636.064941        4.585664
BFGS:    9 18:36:59    -3653.512695        3.840438
BFGS:   10 18:37:01    -3665.206787        7.465894
BFGS:   11 18:37:04    -3674.994385        5.969211
BFGS:   12 18:37:06    -3682.318604        6.232806
BFGS:   13 18:37:08    -3688.445312        2.745205
BFGS:   14 18:37:10    -3693.501953        3.009447
BFGS:   15 18:37:12    -3698.472168        3.233088
BFGS:   16 18:37:15    -3703.159180        2.224850
BFGS:   17 18:37:17    -3706.545898        1.696418
BFGS:   18 18:

In [38]:
# visualize the traj
traj_path = '/Users/sergiortizropero/TFG_phys/NNPs_TFG/COSAN/simulations/water_mds/solvated_ORB_0.0/solvated_ORB_0.0_1.traj'

load_traj = Trajectory(traj_path, 'r')
view(load_traj, viewer='ase')

<Popen: returncode: None args: ['/Users/sergiortizropero/miniconda3/envs/ASE...>

Traceback (most recent call last):
  File "/Users/sergiortizropero/miniconda3/envs/ASE_orbital/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/sergiortizropero/miniconda3/envs/ASE_orbital/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/sergiortizropero/miniconda3/envs/ASE_orbital/lib/python3.10/site-packages/ase/gui/pipe.py", line 32, in <module>
    main()
  File "/Users/sergiortizropero/miniconda3/envs/ASE_orbital/lib/python3.10/site-packages/ase/gui/pipe.py", line 28, in main
    plt.show()
  File "/Users/sergiortizropero/miniconda3/envs/ASE_orbital/lib/python3.10/site-packages/matplotlib/pyplot.py", line 614, in show
    return _get_backend_mod().show(*args, **kwargs)
  File "/Users/sergiortizropero/miniconda3/envs/ASE_orbital/lib/python3.10/site-packages/matplotlib_inline/backend_inline.py", line 90, in show
    display(
  File "/Users/sergiortizropero/miniconda3/env